# Answering Business Questions using SQL

## Introduction

In this project, we will answer a variety of business questions by using SQL to query a SQLite database file, `chinook.db`. The Chinook database contains information on a fictional digital music store such as employees, customers, tracks/artists and sales.

![Chinook Database Schema](images/chinook-schema.svg)


The above schema diagram shows the tables within the Chinook database and how they are connected.

**There are 11 tables in the Chinook database.**

1. **employee** table stores employees data such as employee id, last name, first name, etc. It also has a field named reports_to to specify who reports to whom.
2. **customer** table stores customers data.
3. **invoice** table stores invoice header data
4. **invoice_line** table stores the invoice <u title="The term “line item” refers to any service or product added to an invoice, along with any quantities, rates, and prices that pertain to them.">line items data</u>.
5. **artist** table stores artists data. It is a simple table that contains only artist_id and name.
6. **album** table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
7. **media_type** table stores media types such as MPEG audio and AAC audio file.
8. **genre** table stores music types such as rock, jazz, metal, etc.
9. **track** table store the data of songs. Each track belongs to one album.
10. **playlist** table store data about playlists. Each playlist contains a list of tracks. Each track may belong to multiple playlists. The relationship between the playlist table and track table is <u title="A many-to-many relationship refers to a relationship between tables in a database when a parent row in one table contains several child rows in the second table, and vice versa.">many-to-many</u>.
11. **playlist_track** table is used to reflect this relationship.


## Creating Helper Functions

To help us with our visualisations, we will use `pandas` to write queries that produce dataframe outputs. The advantage of dataframe outputs is that they will display as tables in Jupyter notebook.

We'll also create some helper functions in python to save time and use a context manager to handle the connection to the SQLite database. 

In [1]:
import sqlite3
import pandas as pd

# A function that takes a SQL query as an argument and returns a pandas dataframe.
def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)
    
# A function that takes a SQL command as an argument and executes it using the sqlite module.
def run_command(c):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn, execute(c)
        
# A function that calls the run_query() function to return a list of all tables and views in the database.
def show_tables():
    q = '''
    SELECT name,
           type 
      FROM sqlite_master 
     WHERE type IN ("table", "view");
    '''
    return run_query(q)
    
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four.
All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce.


| Artist Name | Genre |
|-------------|-------|
| Regal       | Hip-Hop|
| Red Tone    | Punk   |
| Meteor and the Girls| Pop |
| Slim Jim Bites| Blues |


The Label specialises in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [2]:
# A query that returns each genre in the Chinook database, with the number of tracks sold in the USA.
usa_genre_sales = '''
WITH usa_track_sales AS
     (
      SELECT i.billing_country AS country,
             il.track_id
        FROM invoice AS i
       INNER JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
       WHERE billing_country = 'USA'
      )
     
SELECT g.name AS genre,
       COUNT(uts.track_id) AS tracks_sold,
       CAST(COUNT(uts.track_id) AS FLOAT) / (
                                             SELECT COUNT(*)
                                               FROM usa_track_sales
                                             ) AS percentage_sold
  FROM track AS t
 INNER JOIN genre AS g ON t.genre_id = g.genre_id
 INNER JOIN usa_track_sales AS uts ON t.track_id = uts.track_id
 GROUP BY genre
 ORDER BY tracks_sold DESC
 LIMIT 10;
'''

run_query(usa_genre_sales)


,genre,tracks_sold,percentage_sold
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Pop,22,0.020932
7,Latin,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


After our analysis, we can see that by far the top selling genre in the USA is rock, with over 53.3% of the total sales.
Unfortunately, none the 4 artists we have to pick from produce rock music, so we will need to take a look at how each of their genres rank.

`Red Tone` produces punk music, which comes in second place in our rankings with over 12.3% of the total sales.
Blues (3.4%) and pop (2.0%) have much less impressive stats, however, they still account for more sales than hip-hop (1.9%).

Therefore, we shall pick `Red Tone`, `Meteor and the Girls` and `Slim Jim Bites` for our store.
We shall discard `Regal` as they produce hip-hop.

## Analysing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase.
Let's analyse the purchases of customers belonging to each employee to see if any sales support agent is under or over performing.

In [3]:
support_agent_sales = '''
SELECT e.first_name || ' ' || e.last_name AS name,
       e.hire_date,
       SUM(i.total) AS total_dollar_sales
  FROM employee AS e
  LEFT JOIN customer AS c ON e.employee_id = c.support_rep_id
  LEFT JOIN invoice AS i ON c.customer_id = i.customer_id
 WHERE title = 'Sales Support Agent'
 GROUP BY name
 ORDER BY total_dollar_sales DESC;
'''

run_query(support_agent_sales)

,name,hire_date,total_dollar_sales
0,Jane Peacock,2017-04-01 00:00:00,1731.51
1,Margaret Park,2017-05-03 00:00:00,1584.00
2,Steve Johnson,2017-10-17 00:00:00,1393.92


While there is a 20% difference between the highest (Jane) and lowest (Steve) sales, this roughly corresponds to the dates which they started.
So, we can see that employee performance is fairly consistent between staff.

## Analysing Sales by Country

Next, let's analyse the sales data for customers from each different country.
In particular, we'll calculate data for each country on the:

* Total number of customers
* Total value of sales
* Average value of sales per customer
* Average order value

As there are a number of countries with only one customer, we'll group these as "Other" in our analysis and put them on the bottom of the results table.


In [4]:
sales_by_country = '''
WITH sales AS
     (
      SELECT c.customer_id,
             c.country,
             i.total AS order_value
        FROM customer AS c
       INNER JOIN invoice AS i ON c.customer_id = i.customer_id
      ),
     sales_per_customer AS
     (
      SELECT customer_id,
             country AS customer_country,
             SUM(order_value) AS total_spent
        FROM sales
       GROUP BY customer_id
      ),
     country_sales AS
     (
      SELECT country,
             COUNT(*) AS total_sales
        FROM sales
       GROUP BY country
      ),
     country_summaries AS
     (
      SELECT country,
             COUNT(customer_id) AS total_customers,
             SUM(total_spent) AS total_value,
             SUM(total_spent) / COUNT(customer_id) AS avg_customer_value,
             SUM(total_spent) / cs.total_sales AS avg_order_value,
             CASE
               WHEN COUNT(customer_id) = 1 THEN 'Other'
               ELSE country
             END AS country_name
        FROM sales_per_customer AS spc
       INNER JOIN country_sales AS cs ON spc.customer_country = cs.country
       GROUP BY country
      )

SELECT country_name,
       SUM(total_customers) AS total_customers,
       SUM(total_value) AS total_value,
       AVG(avg_customer_value) AS avg_customer_value,
       AVG(avg_order_value) AS avg_order_value
  FROM (SELECT *,
               CASE
                 WHEN country_name = 'Other' THEN 0
                 ELSE total_value
               END AS sort
          FROM country_summaries)
 GROUP BY country_name
 ORDER BY sort DESC;
'''

run_query(sales_by_country)

,country_name,total_customers,total_value,avg_customer_value,avg_order_value
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,Czech Republic,2,273.24,136.620000,9.108000
6,United Kingdom,3,245.52,81.840000,8.768571
7,Portugal,2,185.13,92.565000,6.383793
8,India,2,183.15,91.575000,8.721429
9,Other,15,1094.94,72.996000,7.445071


Looking at the resulting data, the following countries may have potential for growth:

* Czech Republic
* United Kingdom
* India

These all have relatively high average customer value but low numbers of total customers, suggesting that the markets have not been fully exploited and still have a lot of potential.
However, the sample size of our data is very small. It would be a good idea to run small campaigns in these countries to gather more data.

## Albums vs Individual Tracks

The Chinook store is setup in a way that allows customers to make purchases in one of two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks

The store does not let customers purchase a whole album, and then add individual tracks to the same purchase.
When customers purchase albums, they are charged the same price as if if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums.

In [5]:
albums_vs_tracks = '''
WITH invoice_first_track AS
     (
      SELECT il.invoice_id,
             MIN(il.track_id) AS track_id
        FROM invoice_line AS il
       GROUP BY invoice_id
      )


SELECT album_purchase,
       COUNT(invoice_id) AS total,
       CAST(COUNT(invoice_id) AS FLOAT) / (
                            SELECT COUNT(*)
                              FROM invoice)
  FROM
       (
        SELECT ift.*,
               CASE
                 WHEN
                      (
                       SELECT t.track_id -- a table of track IDs for all tracks within each album that matches with ifs
                         FROM track AS t
                        WHERE t.album_id = (
                                            SELECT t2.album_id
                                              FROM track AS t2
                                             WHERE t2.track_id = ift.track_id
                                            ) -- a table of album IDs that match with the tracks in ift
               
                       EXCEPT
               
                       SELECT il2.track_id -- a table of all track IDs from each invoice
                         FROM invoice_line AS il2
                        WHERE il2.invoice_id = ift.invoice_id 
                       ) IS NULL
                      AND
                      (
                       SELECT il2.track_id -- a table of all track IDs from each invoice
                         FROM invoice_line AS il2
                        WHERE il2.invoice_id = ift.invoice_id 
               
                       EXCEPT
               
                       SELECT t.track_id -- a table of track IDs for all tracks within each album that matches with ifs
                         FROM track AS t
                        WHERE t.album_id = (
                                            SELECT t2.album_id
                                              FROM track AS t2
                                             WHERE t2.track_id = ift.track_id
                                            ) -- a table of album IDs that match with the tracks in ift
                       ) IS NULL
                 THEN 'yes'
                 ELSE 'no'
                  END AS album_purchase
          FROM invoice_first_track AS ift
        )
 GROUP BY album_purchase;
      
'''

run_query(albums_vs_tracks)

,album_purchase,total,CAST(COUNT(invoice_id) AS FLOAT) / (\n SELECT COUNT(*)\n FROM invoice)
0,no,500,0.814332
1,yes,114,0.185668


Looking at our results, I would not recommend that the store begins a strategy of only buying the most popular tracks from albums as album sales still account for a fifth of their overall sales.